## Investigating Overfitting

### Size of val_loss

In [6]:
import torch
import math
import os

from neural_lam.metrics import wmse


In [11]:
batch_size, pred_steps, n_grid, d_f = 4, 28, 3000, 128
y_hat = torch.rand((batch_size, pred_steps, n_grid, d_f))
y = torch.rand((batch_size, pred_steps, n_grid, d_f))
pred_std = torch.tensor([1])

In [18]:
loss1 = torch.mean(wmse(y_hat, y, pred_std))
loss2 = torch.mean(wmse(y_hat, y, pred_std), dim=0)
loss3 = torch.mean(loss2)
print(loss1.shape)
print(loss2.shape)
print(loss3.shape)


torch.Size([])
torch.Size([28])
torch.Size([])


In [19]:
torch.isclose(loss1, loss3).all()

tensor(True)

### Number of files in train vs val

In [17]:
samples_dir = "/vol/bitbucket/bet20/neural-lam/data/era5_uk/samples/train"
num_files = len(os.listdir(samples_dir))
print(num_files)

1460


In [13]:
total_days = 31 * 7 + 28 + 30 * 4 # Jan, Mar, May, Jul, Aug, Oct, Dec
total_time_steps = total_days * 4
pred_length = 1
sample_length = pred_length + 2
total_samples = total_time_steps - sample_length + 1
print(total_days, total_time_steps, total_samples)

total_batches = math.ceil(total_samples / 4) 
print(total_batches)

365 1460 1458
365


In [21]:
samples_dir = "/vol/bitbucket/bet20/neural-lam/data/era5_uk/samples/val"
dirs = os.listdir(samples_dir)
num_files = len(dirs)
print(dirs)
print(num_files)

total_files = 0
for m in ['01', '04', '07', '10']:
    total_files += len(os.listdir(os.path.join(samples_dir, m)))
    
print(total_files)


['01', '10', '04', 'other', '07']
5
492


In [24]:
total_days = 31 + 30 + 31 + 31
total_time_steps = total_days * 4
pred_length = 28
sample_length = pred_length + 2
total_samples = total_time_steps - sample_length + 1
print(total_days, total_time_steps, total_samples)

total_batches = math.ceil(total_samples / 4) 
print(total_batches)

123 492 463
116


In [25]:
def steps(total_days):
    total_time_steps = total_days * 4
    pred_length = 28
    sample_length = pred_length + 2
    total_samples = total_time_steps - sample_length + 1
    # total_batches = math.ceil(total_samples / 4)
    return total_samples

days = [31, 30, 31, 31]
total_samples = sum([steps(d) for d in days])
total_batches = math.ceil(total_samples / 4)
print(total_batches)

94


## Junk

In [ ]:
dataset_path = "data/era5_uk_reduced/samples/train"
nc_files = glob.glob(f'{constants.DATASET_PATH}/global/*.nc')
print(nc_files)
data = xr.open_dataset(nc_files[0])

In [ ]:
data

In [ ]:
from era5_data_proc import uk_subset
uk_data = uk_subset(data)
uk_data

In [ ]:
uk_data['longitude'] = xr.where(uk_data['longitude'] > 180, uk_data['longitude'] - 360, uk_data['longitude'])

In [ ]:
uk_data

In [ ]:
uk_data.isel(time=0).to_array().values.shape

In [ ]:
latitude = uk_data.latitude.values
latitude

In [ ]:
longitudes = uk_data.longitude.values
longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
longitudes

In [ ]:
latitudes = uk_data.latitude.values
longitudes = uk_data.longitude.values
# print(latitudes.shape, longitudes.shape)

t_lat = torch.from_numpy(latitudes)
t_lon = torch.from_numpy(longitudes)

lat_lon_grid = torch.stack(
    torch.meshgrid(t_lat, t_lon, indexing="ij"), dim=-1
).permute(2, 1, 0)

lat_lon_grid.shape # (2, lon, lat) or (2, x, y)

grid_array = lat_lon_grid.numpy()
grid_array.shape
# np.save("nwp_xy.npy", grid_array)

In [ ]:
np.load("data/era5_uk_reduced/static/nwp_xy.npy")

In [ ]:
processed_data_path="data/era5_uk_reduced/samples/train"
sample_name="20220101000000"
sample_path=f"{processed_data_path}/{sample_name}.npy"

sample=np.load(sample_path)

In [ ]:
import torch

In [ ]:
# lat, lon
input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
latitudes

In [ ]:
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
longitudes

In [ ]:
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)

In [ ]:
lat_lon_grid.shape

In [ ]:
lat_lon_grid

In [ ]:
torch.from_numpy(np.load("data/era5_global/static/nwp_xy.npy")).permute(2, 1, 0)

In [ ]:
print(lat_lon_grid.shape)

In [ ]:
tmp = lat_lon_grid.permute(2, 0, 1)
tmp.shape

In [ ]:
tmp[1][0].shape

In [ ]:
tmp.view(2, -1).shape

In [ ]:
import json
import numpy as np

icospheres_path = "data/era5_global/icospheres.json"
with open(icospheres_path, "r") as f:
    loaded_dict = json.load(f)
    icospheres = {
        key: (np.array(value) if isinstance(value, list) else value)
        for key, value in loaded_dict.items()
    }

In [ ]:
for k in icospheres.keys():
    print(k)

In [ ]:
icospheres["order_0_face_centroid"]

In [ ]:
from graphcast_utils import latlon2xyz
from sklearn.neighbors import NearestNeighbors

input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)
lat_lon_grid_flat = lat_lon_grid.permute(2, 0, 1).view(2, -1).permute(1, 0)
lat_lon_grid_flat.shape

In [ ]:
cartesian_grid = latlon2xyz(lat_lon_grid_flat) # (lat*lon, 3)
n_nbrs = 4
neighbors = NearestNeighbors(n_neighbors=n_nbrs).fit(
    icospheres["order_" + str(6) + "_vertices"] # (40962, 3)
)

In [ ]:
distances, indices = neighbors.kneighbors(cartesian_grid) # (lat*lon, n_nbrs)

In [ ]:
lat_lon_grid_flat

In [ ]:
cartesian_grid

In [ ]:
src, dst = [], []
for i in range(len(cartesian_grid)):
    for j in range(n_nbrs):
        if distances[i][j] <= 0.6 * edge_len:
            src.append(i)
            dst.append(indices[i][j])

In [ ]:
np.load("data/era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 65, 57) (levels, variables, lat, lon)
np.load("data/_era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 721, 1440)

In [ ]:
uk_bbox = {
    "lat_max": 63,
    "lat_min": 47,
    "lon_max": 4,
    "lon_min": -10,
}

torch.linspace(uk_bbox["lat_min"], uk_bbox["lat_max"], steps=65) # (65,)

In [ ]:
torch.linspace(uk_bbox["lon_min"], uk_bbox["lon_max"], steps=57) # (65,)

In [ ]:
import torch

# Assuming larger is your larger tensor and smaller is your smaller tensor
larger = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
smaller = torch.tensor([[1, 2], [7, 8]])

# Expand dimensions for broadcasting
larger_exp = larger.unsqueeze(1)
smaller_exp = smaller.unsqueeze(0)

# Compare all pairs of rows
matches = torch.all(larger_exp == smaller_exp, dim=-1)

# Get indices of matching rows
indices = torch.where(matches)

In [ ]:
indices

In [ ]:
import torch

# Example tensors
larger_tensor = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
smaller_tensor = torch.tensor([[5.0, 6.0], [3.0, 4.0], [1.0, 2.0]])

def find_subset_indices(larger, smaller):
    """
    finds the indices of the smaller tensor in the larger tensor
    """
    indices = []
    for i in range(smaller.size(0)):
        index = torch.where(torch.all(larger == smaller[i], dim=1))[0]
        if index.size(0) == 0:
            raise ValueError("The smaller tensor is not a subset of the larger tensor")
        indices.append(index.item())
    return indices

indices = find_subset_indices(larger_tensor, smaller_tensor)
print(indices)

In [ ]:
import torch
import numpy as np

# Assuming tensor1 and tensor2 are your tensors
tensor1 = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
tensor2 = torch.tensor([[1, 2], [7, 8], [9, 10]])

# Convert tensors to numpy arrays
array1 = tensor1.numpy()
array2 = tensor2.numpy()

# Find common rows
common_rows = np.intersect1d(array1, array2, axis=0)

# Convert back to tensor
common_rows_tensor = torch.from_numpy(common_rows)

In [ ]:
a = list(range(10))
b = [3, 2 , 1]

a[b]

In [ ]:
import neural_lam.constants as constants
w_dict = {
    "2": 1.0,
    "0": 0.1,
    "65": 0.065,
    "1000": 0.1,
    "850": 0.05,
    "500": 0.03,
}
w_list = np.array(
    [w_dict[par.split("_")[-2]] for par in constants.PARAM_NAMES]
)

w_list

# MEPS Dataset

In [ ]:
import numpy as np
import neural_lam.constants as constants
# Create parameter weights based on height
# based on fig A.1 in graph cast paper
w_dict = {
    "2": 1.0,
    "0": 0.1,
    "65": 0.065,
    "1000": 0.1,
    "850": 0.05,
    "500": 0.03,
}
w_list = np.array(
    [w_dict[par.split("_")[-2]] for par in constants.PARAM_NAMES]
)

In [ ]:
for w in zip(w_list, constants.PARAM_NAMES):
    print(w)

# Test the thing

In [ ]:
from neural_lam.constants import MEPSConstants

MEPSConstants.GRID_SHAPE

# Icospheres

In [ ]:
import numpy as np
import json

icospheres_path = "data/era5_uk/icospheres.json"

with open(icospheres_path, "r") as f:
    loaded_dict = json.load(f)
    icospheres = {
        key: (np.array(value) if isinstance(value, list) else value)
        for key, value in loaded_dict.items()
    }
    print(f"Opened pre-computed graph at {icospheres_path}.")

In [ ]:
for i in icospheres.keys():
    print(i)

In [ ]:
len([key for key in icospheres.keys() if "faces" in key]) - 2

In [ ]:
1440 * 720

In [ ]:
import torch

a = torch.tensor([1, 2, 3, 4, 5])
b = torch.tensor([6, 7, 8, 9, 10])

torch.stack((a, b))

# DGL

In [ ]:
from typing import List

import dgl
from dgl import DGLGraph
import torch
import numpy as np
 

def create_heterograph(
    src: List, dst: List, labels: str, dtype: torch.dtype = torch.int32
) -> DGLGraph:
    """Creates a heterogeneous DGL graph from an adj matrix in COO format.

    Parameters
    ----------
    src : List
        List of source nodes
    dst : List
        List of destination nodes
    labels : str
        Label of the edge type
    dtype : torch.dtype, optional
        Graph index data type, by default torch.int32

    Returns
    -------
    DGLGraph
        The dgl Graph.
    """
    graph = dgl.heterograph({labels: ("coo", (src, dst))}, idtype=dtype)
    return graph

src = [np.array([0, 1, 2]), np.array([3, 4, 5])]
dst = [np.array([1, 2, 3]), np.array([4, 5, 6])]

graph = create_heterograph(src, dst, ("mesh", "m2g", "grid"), dtype=torch.int32)


In [ ]:
graph

# Ridiculous

In [ ]:
"""
           5663073       gpu submit_j    bet20 PD       0:00      1 (Priority)
           5663071       gpu submit_j    bet20 PD       0:00      1 (Resources)
     5662324_[0-4]       gpu    F2mIF  qwang33 PD       0:00      1 (Priority)
     5662323_[1-5]       gpu    F2mIF  qwang33 PD       0:00      1 (Priority)
           5660755       gpu 24layers   cc2040 PD       0:00      2 (Priority)
           5660861       gpu    r-c16 yiweiche PD       0:00      1 (Priority)
           5660862       gpu    r-c32 yiweiche PD       0:00      1 (Priority)
           5660900       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660898       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660899       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660897       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660893       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660894       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660895       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660896       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660889       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660890       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660891       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660892       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660888       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660887       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660886       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662524       gpu sr327mod gpw2-ec2 PD       0:00      4 (Priority)
           5662502       gpu    uw_G1   aa2201 PD       0:00      1 (Priority)
           5662706       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662707       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662704       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662705       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662702       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662703       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662701       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662699       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662700       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662697       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662698       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662696       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662694       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662695       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662692       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662693       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662691       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662689       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662690       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662687       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662688       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662686       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662684       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662685       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662682       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662683       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662681       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662679       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662680       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662677       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662678       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662675       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662676       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662671       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662672       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662673       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662674       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662669       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662670       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662665       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662666       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662667       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662668       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662663       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662664       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662659       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662660       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662661       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662662       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662657       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662658       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662656       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662653       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662654       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662655       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662651       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662652       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662648       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662649       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662650       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662646       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662647       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662643       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662644       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662645       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662642       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662641       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662638       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662639       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662640       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662635       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662636       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662637       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662633       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662634       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662631       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662632       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662629       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662630       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662627       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662628       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662626       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662625       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662624       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662623       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662622       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662621       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662620       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662619       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662618       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662617       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662616       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662615       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662614       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662613       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662612       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662611       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662610       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662609       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662608       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5663082       gpu uw_G1_ra   aa2201 PD       0:00      1 (Priority)
           5663081       gpu bio512_2   aa2201 PD       0:00      1 (Priority)
           5663080       gpu   bio512   aa2201 PD       0:00      1 (Priority)
           5663078       gpu ast512_2   aa2201 PD       0:00      1 (Priority)
           5663076       gpu   ast512   aa2201 PD       0:00      1 (Priority)
           5662472       gpu   MUMAX3      cpy PD       0:00      1 (Priority)
           5662195       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662191       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662192       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662193       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662194       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662190       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662186       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662187       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662188       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662189       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662182       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662183       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662184       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662185       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662201       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662200       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662199       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662198       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662197       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662196       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662473       gpu   MUMAX3      cpy PD       0:00      1 (Dependency)
           5662598       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662597       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662596       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662595       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662594       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662593       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662592       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662591       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662590       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662589       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5631081       gpu HemKtest   julian  R 13-03:43:08      1 r2i6n0
           5662497       gpu run_XCLI xingran0  R    4:51:28      4 r2i4n[1-4]
           5660733       gpu     mace juraskov  R   11:50:40      1 r2i5n7
           5660732       gpu     mace juraskov  R   11:59:47      1 r2i7n1
           5660731       gpu     mace juraskov  R   12:05:27      1 r2i5n7
         5660551_5       gpu    F2mIF  qwang33  R   13:54:11      1 r2i6n3
         5660551_4       gpu    F2mIF  qwang33  R   14:21:30      1 r2i4n0
           5660715       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660716       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660717       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660718       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660719       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660720       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660721       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660722       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660723       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660724       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660725       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660726       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660727       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660728       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660729       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660730       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660711       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660712       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660713       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660714       gpu     mace juraskov  R   13:54:11      1 r2i7n3
         5660551_3       gpu    F2mIF  qwang33  R   15:29:46      1 r2i7n0
         5660551_1       gpu    F2mIF  qwang33  R   16:04:11      1 r2i7n2
         5660551_2       gpu    F2mIF  qwang33  R   16:04:11      1 r2i7n4
           5660710       gpu     mace juraskov  R   14:24:38      1 r2i6n0
           5660709       gpu     mace juraskov  R   14:50:53      1 r2i5n6
           5660708       gpu     mace juraskov  R   16:15:23      1 r2i6n0
           5660707       gpu     mace juraskov  R   16:16:24      1 r2i6n0
           5662126       gpu   myvenv zhenyaza  R   11:03:18      1 r2i4n6
           5662127       gpu   myvenv zhenyaza  R   11:03:18      1 r2i5n8
           5660706       gpu     mace juraskov  R   17:55:58      1 r2i7n6
           5660705       gpu     mace juraskov  R   18:04:32      1 r2i7n6
           5660704       gpu     mace juraskov  R   20:16:54      1 r2i6n4
           5660703       gpu     mace juraskov  R   20:18:55      1 r2i6n4
           5660702       gpu     mace juraskov  R   20:26:30      1 r2i5n6
           5660701       gpu     mace juraskov  R   20:33:04      1 r2i5n7
           5660699       gpu     mace juraskov  R   21:40:48      1 r2i7n6
           5660700       gpu     mace juraskov  R   21:40:48      1 r2i7n6
           5660698       gpu     mace juraskov  R   23:17:14      1 r2i4n7
           5656524       gpu     mace juraskov  R 3-21:13:59      1 r2i5n3
           5660697       gpu     mace juraskov  R 1-00:47:30      1 r2i5n6
           5660696       gpu     mace juraskov  R 1-00:50:01      1 r2i5n6
           5660695       gpu     mace juraskov  R 1-01:24:47      1 r2i4n7
           5660694       gpu     mace juraskov  R 1-02:37:16      1 r2i6n4
           5660693       gpu     mace juraskov  R 1-02:37:17      1 r2i6n4
         5662323_0       gpu    F2mIF  qwang33  R    7:29:11      1 r2i5n4
           5660753       gpu sr327mod gpw2-ec2  R    7:51:43      4 r2i6n[6-8],r2i7n5
           5660692       gpu     mace juraskov  R 1-03:30:36      1 r2i4n7
           5660686       gpu     mace juraskov  R 1-03:31:06      1 r2i4n7
           5660687       gpu     mace juraskov  R 1-03:31:06      1 r2i7n1
           5660688       gpu     mace juraskov  R 1-03:31:06      1 r2i7n1
           5660689       gpu     mace juraskov  R 1-03:31:06      1 r2i5n1
           5660690       gpu     mace juraskov  R 1-03:31:06      1 r2i5n1
           5660691       gpu     mace juraskov  R 1-03:31:06      1 r2i5n7
           5660885       gpu     traj xr223_ci  R    5:20:49      1 r2i7n7
           5660884       gpu     traj xr223_ci  R    5:21:19      1 r2i7n7
           5660883       gpu     traj xr223_ci  R    8:02:01      1 r2i7n7
           5660882       gpu     traj xr223_ci  R    8:03:31      1 r2i7n7
           5660881       gpu     traj xr223_ci  R   11:17:41      1 r2i7n1
           5657937       gpu       pd yiweiche  R 3-07:08:49      1 r2i5n1

"""